In [11]:
import yaml
from yaml.loader import SafeLoader
from datetime import datetime
import time
import copy
import pandas as pd
import threading
file='Milestone1B.yaml'

with open(file) as f:
    config = yaml.load(f, Loader=SafeLoader)
print(config)

{'M1B_Workflow': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskA': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}, 'FlowA': {'Type': 'Flow', 'Execution': 'Concurrent', 'Activities': {'TaskB': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '14'}}, 'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '5'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '5'}}, 'FlowB': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskE': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskE_Input', 'ExecutionTime': '5'}}, 'TaskF': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskF_Input', 'ExecutionTime': '1'}}}}, 'FlowC': {'Type': 'Flow', 'Execution': 'Sequential', 'Activ

In [12]:
def TimeFunction(FunctionInput,ExecutionTime):
    print(1)
    time.sleep(int(ExecutionTime))
    print(int(ExecutionTime))
    return
    
def DataLoad(Filename):
    print(2)
    DataTable=pd.read_csv(Filename)
    NoOfDefects=DataTable.shape[0]-1
    
    return DataTable, NoOfDefects


def callfunction(name,inputs):
    if name=='TimeFunction':
        TimeFunction(inputs[0],inputs[1])
    elif name=='DataLoad':
        DataLoad(inputs[0])

In [13]:
def runTask(task,parents):
    print(task)
    name=''
    for i in parents:
        name+=str(i)+'.'
    
    f.write(str(datetime.now())+';'+name[:-1]+' Entry\n')
    if task['Type']=='Task':
        funs=''
        parms=''
        inputs=[]

        for i in task['Inputs'].values():
            inputs.append(i)
            funs+=str(i)+', '
#         if(task['Function']=='TimeFunction'):
#             parms=str(inputs[-1])+','
#         else:
        parms=funs
                
        
        f.write(str(datetime.now())+';'+name[:-1]+' Executing ' +str(task['Function'])+' ('+parms[:-2]+')\n')
        callfunction(task['Function'],inputs)
        
    else:
        if(task['Execution']=='Sequential'):
            Activits=task['Activities']
            for i in Activits:
                par=copy.deepcopy(parents)
                par.append(i)
                runTask(Activits[i],par)
        else:
            Activits=task['Activities']
            threads=[]
            for i in Activits:
                par=copy.deepcopy(parents)
                par.append(i)
                t=threading.Thread(target=runTask, args=(Activits[i],par,))
                threads.append(t)
                t.start()
            for t in threads:
                t.join()
    f.write(str(datetime.now())+';'+name[:-1]+' Exit\n')

In [14]:
task_list=list(config.keys())[0]
f = open(file[:-5]+"_Log.txt", "w")
f.write(str(datetime.now())+';'+task_list+' Entry\n')
baseActivities=config[task_list]['Activities']
for i in baseActivities:
    runTask(baseActivities[i],[task_list,i])

f.write(str(datetime.now())+';'+task_list+' Exit')
f.close()

{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskA_Input', 'ExecutionTime': '1'}}
1
1
{'Type': 'Flow', 'Execution': 'Concurrent', 'Activities': {'TaskB': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '14'}}, 'TaskC': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskC_Input', 'ExecutionTime': '5'}}, 'TaskD': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskD_Input', 'ExecutionTime': '5'}}, 'FlowB': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskE': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskE_Input', 'ExecutionTime': '5'}}, 'TaskF': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskF_Input', 'ExecutionTime': '1'}}}}, 'FlowC': {'Type': 'Flow', 'Execution': 'Sequential', 'Activities': {'TaskG': {'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 